In [ ]:
from ipywidgets import interact, Dropdown, Output, GridspecLayout, VBox, BoundedIntText, HBox, Button, TwoByTwoLayout, Layout
from math import prod

import calculations

In [ ]:
total = [150, 1, 1, 1]
final_shards = Output(outputs=[{'output_type': 'stream', 'text': '150'}])

In [ ]:
### Define the Devourer Level widgets

# Define the options for dropdown menus
options = {
    'Devourer Lvl': [x for x in range(35, 101, 5)]
}

# Define the output textbox
output_level = Output(outputs=[{'output_type': 'stream', 'text': '150'}])

# Define a function to update the level textbox
def update_output_level(dev_level_dropdown_val):
    total[0] = int(calculations.get_level_shards(dev_level_dropdown_val['new']))
    output_level.clear_output()
    with output_level:
        print(f"{calculations.get_level_shards(dev_level_dropdown_val['new']):,}")    # :, automatically adds the thousands separator
    with final_shards:
        final_shards.clear_output()
        print(f'{int(prod(total)):,}')

# Create dropdown widgets
dev_level_dropdown = Dropdown(options=options['Devourer Lvl'], description='Devourer Lvl:')

# Create an interaction between the dropdowns and the output textbox
dev_level_dropdown.observe(update_output_level, names='value')

In [ ]:
### Define the Feats widgets

# Define the options for dropdown menus
options = {
    'Feats': [x for x in range(1, 26)],
}

# Define the output textbox
output_feats = Output(outputs=[{'output_type': 'stream', 'text': '100%'}])

# Define a function to update the feats textbox
def update_output_feats(feats_dropdown_val):
    total[1] = int(calculations.get_feat_shards(feats_dropdown_val['new'])) / 100.
    output_feats.clear_output()
    with output_feats:
        print(f"{calculations.get_feat_shards(feats_dropdown_val['new'])}%")
    with final_shards:
        final_shards.clear_output()
        print(f'{int(prod(total)):,}')

# Create dropdown widgets
feats_dropdown = Dropdown(options=options['Feats'], description='Feats:')

# Create an interaction between the dropdowns and the output textbox
feats_dropdown.observe(update_output_feats, names='value')

In [ ]:
### Define the Other widget

# Define the output textbox
output_other = Output(outputs=[{'output_type': 'stream', 'text': '100%'}])

# Define int input box for Other category
other_shards = BoundedIntText(value=100, min=100, max=150, description='Other:', disabled=False)

# Define a function to update the other textbox
def update_output_other(other_val):
    total[3] = int(other_val['new']) / 100.
    output_other.clear_output()
    with output_other:
        print(f"{other_val['new']}%")
    with final_shards:
        final_shards.clear_output()
        print(f'{int(prod(total)):,}')

# Create an interaction between the dropdowns and the output textbox
other_shards.observe(update_output_other, names='value')

In [ ]:
### Define the legendaries widget

# Define the output textbox
output_legend = Output(outputs=[{'output_type': 'stream', 'text': '100%'}])

# Define the options for dropdown menus
options = calculations.list_legendary_limits()

# Define the descriptions for dropdown menus
descriptions = calculations.list_legendaries()

# Create a 4x4 grid to hold the dropdowns
grid = GridspecLayout(4, 4)

# Create callback for grid changes
def cell_eventhandler(change):
    output_legend.clear_output()
    with output_legend:
        calculations.update_legendary_count(change['new'], change['owner'].description)
        print(f'{calculations.get_legendary_bonus()}%')
    total[2] = int(calculations.get_legendary_bonus()) / 100.
    with final_shards:
        final_shards.clear_output()
        print(f'{int(prod(total)):,}')

# Create callback for clear button
def on_clear_click(click):
    calculations.init_legendary_table()
    reset_grid()
    

# Reset legendary counts to 0
def reset_grid():
    for i in range(4):
        for j in range(3):
            grid[i, j].value = 0

# Loop through each cell in the grid
for i in range(4):
    for j in range(3):
        idx = (3 * i) + j
        desc = descriptions[idx]
        
        if options[idx] == -1:
            textbox = BoundedIntText(value=0, min=0, max=100, description=f'{desc}', disabled=False)
            textbox.observe(cell_eventhandler, names='value')
            grid[i, j] = textbox
        else:
            dropdown = Dropdown(options=[x for x in range(0, options[idx] + 1)], description=f'{desc}')
            dropdown.observe(cell_eventhandler, names='value')
            grid[i, j] = dropdown

# Add output box and clear button in last column
clear_button = Button(description='Clear', disabled=False, layout=Layout(height='auto', width='auto'))
grid[:4, 3] = clear_button

# Add callback to button
clear_button.on_click(on_clear_click)

In [ ]:
### Output cells

outputs = HBox([
    output_level, Output(outputs=[{'output_type': 'stream', 'text': 'x'}]),
    output_feats, Output(outputs=[{'output_type': 'stream', 'text': 'x'}]),
    output_legend, Output(outputs=[{'output_type': 'stream', 'text': 'x'}]),
    output_other, Output(outputs=[{'output_type': 'stream', 'text': '='}]),
    final_shards
])
               

In [ ]:
##### Put it all together
spacer_widget = Output(outputs=[{'output_type': 'stream', 'text': '\n'}])
multi_spacer_widget = Output(outputs=[{'output_type': 'stream', 'text': '\n\n\n'}])

level_and_feat = TwoByTwoLayout(
    top_left=dev_level_dropdown, 
    top_right=output_level,
    bottom_left=feats_dropdown,
    bottom_right=output_feats)
other = HBox([other_shards, output_other])

widget = VBox([dev_level_dropdown, spacer_widget, feats_dropdown, spacer_widget, grid, spacer_widget, other_shards, multi_spacer_widget, outputs])
widget